In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os, sys, re, datetime
from pathlib import Path
from collections import defaultdict

pj_dir = Path(os.getcwd()).parent
data_dir = pj_dir/'data'
src_dir = pj_dir/'src'
sys.path.append(str(src_dir))

from matplotlib import pyplot as plt
plt.style.use("bmh")
import numpy as np
import pandas as pd
import dask.dataframe as dd
from scipy import stats 

from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv(data_dir/'talklist.csv')

### additional_infoに「キャンセルが入っていないものだけ抽出」

In [3]:
df = df[df['additional_info'].isnull()]

### audience_levelを{"All": 0, "Beginner": 1, "Intermediate": 2, "Advanced": 3}で変換

In [5]:
df['level'] = df['audience_level'].replace({"All": 0, "Beginner": 1, "Intermediate": 2, "Advanced": 3})

# 割当アルゴリズム

In [10]:
def assign(df, df_assign):

    assigned_ids = []
    row_genre_dic = defaultdict(list)
    row_lang_dic = defaultdict(list)
    n_row, n_col = df_assign.shape
    for c in range(n_col):
        for r in range(n_row):
            if df_assign.iloc[r, c] == -1:
                print('割当できない時間帯です')
                continue

            target_index = 0
            force_flg = False
            while True:
                df_rest = df.query('id not in @assigned_ids')
                if df_rest.shape[0] == 0:
                    print('割当てるトークがありません')
                    df_assign.iloc[r, c] = -1
                    break
                try:
                    target_s = df_rest.iloc[target_index]
                except IndexError:
                    print('割当できなかったので強制割当します')
                    target_index = 0
                    force_flg = True

                target_id = target_s['id']
                target_genre = target_s['genre']
                target_lang = target_s['lang']

                cond1 = target_genre in row_genre_dic[r]
                cond2 = row_lang_dic[r].count(target_lang) > int(n_col/2)

                if not force_flg and (cond1 or cond2) :
                    target_index += 1
                    continue

                assigned_ids.append(target_id)
                row_genre_dic[r].append(target_genre)
                row_lang_dic[r].append(target_lang)
                desc = describe(target_s)
                df_assign.iloc[r, c] = desc
                print("割り当てました", desc)
                break
    
    df_rest = df.query('id not in @assigned_ids')
    if df_rest.shape[0] > 0:
        print("割当てられていないトークがあります")
        for i, r in df.iterrows():
            print(describe(r))
        
        
    return df, df_assign

# ユーティリティ

In [8]:
def describe(s):
    title = s['title']
    id_ = s['id']
    genre = int(s['genre'])
    level = int(s['level'])
    lang = s['lang']
    
    desc = "{}:G{}_L{}_{}_{}".format(id_, genre, level, lang, title)
    return desc

# 45分枠

In [6]:
df45 = df.query('talk_format == "Talk (45 minutes)"').copy()

### ソート

In [346]:
df45.sort_values(['level', 'rating'], ascending=[True, False], inplace=True)

### 枠を準備

In [38]:
df_assign45 = pd.DataFrame(np.zeros((4, 7)))
df_assign45.iloc[0:3, 2] = -1
df_assign45.iloc[0:2, 3] = -1

In [37]:
df_assign45

,0,1,2,3,4,5,6
0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0
1,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0
2,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df45, df_assign45 = assign(df45, df_assign45)

割り当てました 4:G0_L0_en_Build text classification models ( CBOW and Skip-gram) with FastText in python
割り当てました 8:G0_L0_en_Probabilistic Programming and Bayesian Deep Learning (comparing Edward and ZhuSuan python libraries)
割り当てました 9:G4_L0_ja_Pythonを使ったハードウェア開発について
割り当てました 10:G0_L3_ja_実践・競馬データサイエンス
割り当てました 11:G4_L1_en_HomeSecurity with Python
割り当てました 12:G4_L1_ja_あなたと私いますぐパッケージン
割り当てました 13:G1_L0_ja_DjangoではじめるPyCharm実践入門
割り当てました 14:G2_L0_ja_「リモートペアプロでマントルを突き抜けろ！」AWS Cloud9でリモートペアプロ＆楽々サーバーレス開発
割当できない時間帯です
割当できない時間帯です
割当できない時間帯です
割り当てました 15:G1_L2_en_Integrate Full-text Search service with Django
割当できない時間帯です
割当できない時間帯です
割り当てました 16:G3_L2_ja_JVM上で動くPython3処理系cafebabepyの実装詳解
割り当てました 22:G4_L0_ja_PyCon JP における子ども向けワークショップの活動事例と実施の意義
割り当てました 17:G1_L1_ja_Djangoだってカンバンつくれるもん(Django Channels + Vue)
割り当てました 18:G2_L2_en_Applying serverless architecture pattern to distributed data processing
割り当てました 19:G2_L1_ja_Pythonistaに贈るコンテナ入門
割り当てました 26:G3_L0_en_Migrating from Py2 application to Py3: first trial in Mon

In [44]:
df_assign45

,0,1,2,3,4,5,6
0,4:G0_L0_en_Build text classification models ( ...,11:G4_L1_en_HomeSecurity with Python,-1,-1,17:G1_L1_ja_Djangoだってカンバンつくれるもん(Django Channel...,"28:G3_L0_ja_オンザフライ高速化パッケージの比較：Numba, TensorFlo...",25:G4_L0_ja_Sphinx-2.0 とドキュメントの未来
1,8:G0_L0_en_Probabilistic Programming and Bayes...,12:G4_L1_ja_あなたと私いますぐパッケージン,-1,-1,18:G2_L2_en_Applying serverless architecture p...,20:G1_L2_ja_Django REST Framework におけるAPI実装プラクティス,24:G0_L0_ja_niconicoにおけるコンテンツレコメンドの取り組み
2,9:G4_L0_ja_Pythonを使ったハードウェア開発について,13:G1_L0_ja_DjangoではじめるPyCharm実践入門,-1,16:G3_L2_ja_JVM上で動くPython3処理系cafebabepyの実装詳解,19:G2_L1_ja_Pythonistaに贈るコンテナ入門,29:G3_L0_en_Why you should care about types: P...,23:G0_L2_ja_SymPyによる数式処理
3,10:G0_L3_ja_実践・競馬データサイエンス,14:G2_L0_ja_「リモートペアプロでマントルを突き抜けろ！」AWS Cloud9でリ...,15:G1_L2_en_Integrate Full-text Search service...,22:G4_L0_ja_PyCon JP における子ども向けワークショップの活動事例と実施の意義,26:G3_L0_en_Migrating from Py2 application to ...,27:G1_L1_ja_Webアプリケーションの仕組み,21:G1_L0_ja_Djangoアプリケーションにおけるトイル撲滅戦記


In [45]:
df_assign45.to_clipboard(sep=',', index=False)

# 30分枠

In [14]:
df30 = df.query('talk_format == "Talk (30 minutes)"').copy()
df30.sort_values(['level', 'rating'], ascending=[True, False], inplace=True)

### 枠を準備

In [27]:
df_assign30 = pd.DataFrame(np.zeros((5, 6)))

In [28]:
df30, df_assign30 = assign(df30, df_assign30)

割り当てました 49:G4_L0_en_Fun with Python and Kanji
割り当てました 5:G0_L0_en_Creative Music Applications in Python
割り当てました 41:G4_L0_en_Make a Drone using RaspberryPi and Google VoiceKit by Python
割り当てました 57:G0_L0_ja_1次元畳み込みフィルターを利用した音楽データのオートエンコーダ
割り当てました 37:G0_L0_en_Visualizing Topic Models
割り当てました 52:G0_L0_en_Detecting offensive messages using Deep Learning: A micro-service based approach
割り当てました 36:G2_L0_ja_Jupyterで広がるPythonの可能性
割り当てました 56:G1_L0_en_Adding JWT Authentication to Python and Django REST Framework Using Auth0
割り当てました 34:G4_L0_ja_Pythonで解く大学入試数学
割り当てました 7:G1_L0_ja_Pythonで始めるウェブスクレイピング実践入門
割り当てました 30:G2_L0_en_Python, AWS and FinTech
割り当てました 31:G4_L0_ja_Python研修の作り方-Teaching Is Learning-
割り当てました 55:G0_L0_ja_メルカリにおける AI 活用事例
割り当てました 44:G3_L1_ja_Pythonでざっくり学ぶUnixプロセス
割り当てました 54:G4_L1_ja_AltJSとしてのPython - フロントエンドをPythonで書こう
割り当てました 6:G1_L1_ja_自分が欲しいものをPythonで書く方法(Python for Myself)
割り当てました 35:G1_L1_en_Notebook as Web API: Turn your notebook into Web API
割り当てました 42:G2_L2_ja_複数アプリケーションのプロセス

In [29]:
df_assign30

,0,1,2,3,4,5
0,49:G4_L0_en_Fun with Python and Kanji,52:G0_L0_en_Detecting offensive messages using...,"30:G2_L0_en_Python, AWS and FinTech",6:G1_L1_ja_自分が欲しいものをPythonで書く方法(Python for Mys...,53:G3_L3_ja_C拡張と共に乗り切るPython 2→3移行術,39:G1_L2_en_The Modern OAuth 2.0
1,5:G0_L0_en_Creative Music Applications in Python,36:G2_L0_ja_Jupyterで広がるPythonの可能性,31:G4_L0_ja_Python研修の作り方-Teaching Is Learning-,35:G1_L1_en_Notebook as Web API: Turn your not...,32:G4_L2_ja_Pythonによる異常検知入門,43:G0_L2_ja_Pythonistaの選球眼（せんきゅうがん） - エンジニアリング...
2,41:G4_L0_en_Make a Drone using RaspberryPi and...,56:G1_L0_en_Adding JWT Authentication to Pytho...,55:G0_L0_ja_メルカリにおける AI 活用事例,42:G2_L2_ja_複数アプリケーションのプロセスとログを管理するための新しいツールと手法,33:G0_L2_ja_料理写真が美味しく撮れる！ 開発現場から覗くAI料理カメラの裏側,"45:G0_L1_nan_Interpretable Machine Learning, m..."
3,57:G0_L0_ja_1次元畳み込みフィルターを利用した音楽データのオートエンコーダ,34:G4_L0_ja_Pythonで解く大学入試数学,44:G3_L1_ja_Pythonでざっくり学ぶUnixプロセス,50:G1_L2_en_From Data to Web Application: Anim...,48:G0_L2_en_How to Data Wrangling? Tips for us...,40:G0_L1_ja_Pythonで「お絵描きパズル」を解いてみた。
4,37:G0_L0_en_Visualizing Topic Models,7:G1_L0_ja_Pythonで始めるウェブスクレイピング実践入門,54:G4_L1_ja_AltJSとしてのPython - フロントエンドをPythonで書こう,51:G2_L2_ja_Django を Zappaで構築してServerless Pyth...,38:G4_L2_ja_WILDCAT SDKは量子コンピュータビジネスの味方となるのか！？,47:G0_L1_ja_Interactive Network Visualization ...


In [30]:
df_assign30.to_clipboard(sep=',', index=False)